In [1]:
import time
import datetime
import os
from Research.backtest.bt import BTDaily
import matplotlib.pyplot as plt
from Research.feature.ft import FeatureAnalysis
import Research.utils.namespace as namespace
import Research.utils.normalization as norm
from Platform.database.mysql import MysqlAPI
from Platform.utils.persistence import convert_to_standard_daily_feature_csv, convert_to_standard_daily_feature_par
from Platform.config.mysql_info import FACTOR_LIB_MYSQL_TIO
import DataAPI
from os import walk
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from progressbar import ProgressBar
from tqdm import tqdm
import multiprocessing as mp
import sys
sys.path.append("/home/wuwenjun/jupyter_code/Shannon/AlphaNet/packages/")
import AlphaNet
from AlphaNet.Data import DataLoader
from AlphaNet.Models import AlphaNet_LSTM_V1

Loading the configuration from /home/ShareFolder/lgc/Modules/Research/config/feature_bt_template


In [2]:
data_path = "/home/wuwenjun/Data/"
loader1 = DataLoader()
loader1.load_data_from_file(alpha_name = "AlphaNetV4_Moving10_Shift_1_Sequence_20_10dRet",end_date = "2022-01-01",data_path=data_path)

100%|██████████| 1705/1705 [07:46<00:00,  3.65it/s]


COV_div_open_close_div_high_close_10_Shift_19  \
timestamp           ticker                                                  
2016-02-19 09:30:00 000001                                       0.000439   
                    000004                                       0.002332   
                    000005                                       0.002369   
                    000006                                       0.002173   
                    000008                                       0.002430   
...                                                                   ...   
2021-05-31 09:30:00 688698                                       0.000441   
                    688777                                       0.000267   
                    688788                                       0.000304   
                    688819                                       0.000078   
                    688981                                       0.000106   

                            COV_div_open_close_div_low_close_10_Shift_19  \
timestamp           ticker                                                 
2016-02-19 09:30:00 000001                                      0.000147   
                    000004                                      0.002056   
                    000005                                      0.001883   
                    000006                                      0.001124   
                    000008                                      0.002762   
...                                                                  ...   
2021-05-31 09:30:00 688698                                      0.000623   
                    688777                                      0.000093   
                    688788                                      0.000476   
                    688819                                      0.000120   
                    688981                                      0.000105   

                            COV_div_open_close_close_10_Shift_19  \
timestamp           ticker                                         
2016-02-19 09:30:00 000001                             -0.062245   
                    000004                              0.061164   
                    000005                             -0.049824   
                    000006                              0.059243   
                    000008                             -0.326933   
...                                                          ...   
2021-05-31 09:30:00 688698                             -0.010271   
                    688777                             -0.027520   
                    688788                             -0.026010   
                    688819                             -0.005039   
                    688981                             -0.006697   

                            COV_div_open_close_volume_sqrt_8_10_Shift_19  \
timestamp           ticker                                                 
2016-02-19 09:30:00 000001                                     -0.002987   
                    000004                                     -0.007366   
                    000005                                     -0.007878   
                    000006                                     -0.003415   
                    000008                                     -0.008294   
...                                                                  ...   
2021-05-31 09:30:00 688698                                     -0.002101   
                    688777                                      0.000337   
                    688788                                     -0.000738   
                    688819                                     -0.000232   
                    688981                                      0.000537   

                            COV_div_open_close_pct_chg_10_Shift_19  \
timestamp           ticker                                           
2016-02-19 09:30:00 000

In [3]:
loader2 = DataLoader()
loader2.load_data_from_file(alpha_name = "AlphaNetV4_Moving5_Shift_1_Sequence_20",end_date = "2022-01-01",data_path=data_path)

100%|██████████| 1705/1705 [06:01<00:00,  4.72it/s]


COV_div_open_close_div_high_close_5_Shift_19  \
timestamp           ticker                                                 
2016-02-05 09:30:00 000001                                      0.000743   
                    000004                                      0.002670   
                    000005                                      0.002917   
                    000006                                      0.002723   
                    000008                                      0.002700   
...                                                                  ...   
2021-05-31 09:30:00 688698                                      0.000259   
                    688699                                      0.000394   
                    688777                                      0.000537   
                    688788                                      0.000016   
                    688981                                      0.000114   

                            COV_div_open_close_div_low_close_5_Shift_19  \
timestamp           ticker                                                
2016-02-05 09:30:00 000001                                     0.000183   
                    000004                                     0.001959   
                    000005                                     0.002290   
                    000006                                     0.001263   
                    000008                                     0.003259   
...                                                                 ...   
2021-05-31 09:30:00 688698                                     0.000337   
                    688699                                     0.001395   
                    688777                                     0.000008   
                    688788                                     0.000025   
                    688981                                     0.000081   

                            COV_div_open_close_close_5_Shift_19  \
timestamp           ticker                                        
2016-02-05 09:30:00 000001                            -0.328205   
                    000004                             0.107023   
                    000005                            -0.124815   
                    000006                            -0.223509   
                    000008                            -0.387713   
...                                                         ...   
2021-05-31 09:30:00 688698                            -0.008291   
                    688699                            -0.762003   
                    688777                            -0.044412   
                    688788                             0.007781   
                    688981                            -0.002644   

                            COV_div_open_close_volume_sqrt_8_5_Shift_19  \
timestamp           ticker                                                
2016-02-05 09:30:00 000001                                    -0.005325   
                    000004                                    -0.014517   
                    000005                                    -0.018106   
                    000006                                    -0.013636   
                    000008                                    -0.018721   
...                                                                 ...   
2021-05-31 09:30:00 688698                                    -0.000696   
                    688699                                    -0.011797   
                    688777                                     0.002380   
                    688788                                     0.000882   
                    688981                                     0.000720   

                            COV_div_open_close_pct_chg_5_Shift_19  \
timestamp           ticker                                          
2016-02-05 09:30:00 000001                              -0.105278   
        

In [4]:
loader3 = DataLoader()
loader3.load_data_from_file(alpha_name = "AlphaNetV4_Moving20_Shift_1_Sequence_20",end_date = "2022-01-01",data_path=data_path)

100%|██████████| 1705/1705 [05:37<00:00,  5.06it/s] 


COV_div_open_close_div_high_close_20_Shift_19  \
timestamp           ticker                                                  
2016-03-04 09:30:00 000001                                       0.000338   
                    000004                                       0.001981   
                    000005                                       0.002123   
                    000006                                       0.001678   
                    000008                                       0.001836   
...                                                                   ...   
2021-05-31 09:30:00 688699                                       0.001102   
                    688777                                       0.000242   
                    688788                                       0.000237   
                    688819                                       0.000177   
                    688981                                       0.000101   

                            COV_div_open_close_div_low_close_20_Shift_19  \
timestamp           ticker                                                 
2016-03-04 09:30:00 000001                                      0.000163   
                    000004                                      0.001492   
                    000005                                      0.001950   
                    000006                                      0.000917   
                    000008                                      0.001779   
...                                                                  ...   
2021-05-31 09:30:00 688699                                      0.001347   
                    688777                                      0.000176   
                    688788                                      0.000307   
                    688819                                      0.000097   
                    688981                                      0.000112   

                            COV_div_open_close_close_20_Shift_19  \
timestamp           ticker                                         
2016-03-04 09:30:00 000001                              0.014728   
                    000004                              0.080382   
                    000005                             -0.039610   
                    000006                              0.012983   
                    000008                             -0.115969   
...                                                          ...   
2021-05-31 09:30:00 688699                             -0.057923   
                    688777                             -0.007302   
                    688788                             -0.016833   
                    688819                             -0.006257   
                    688981                             -0.004703   

                            COV_div_open_close_volume_sqrt_8_20_Shift_19  \
timestamp           ticker                                                 
2016-03-04 09:30:00 000001                                     -0.000982   
                    000004                                     -0.003985   
                    000005                                     -0.003899   
                    000006                                     -0.001042   
                    000008                                     -0.005102   
...                                                                  ...   
2021-05-31 09:30:00 688699                                     -0.000576   
                    688777                                     -0.000988   
                    688788                                     -0.000102   
                    688819                                      0.000539   
                    688981                                     -0.000632   

                            COV_div_open_close_pct_chg_20_Shift_19  \
timestamp           ticker                                           
2016-03-04 09:30:00 000

In [5]:
final_df = pd.concat([loader1.feature_data,loader2.feature_data,loader3.feature_data],axis=1)
final_df

COV_div_open_close_div_high_close_10_Shift_19  \
timestamp           ticker                                                  
2016-02-05 09:30:00 000001                                            NaN   
                    000004                                            NaN   
                    000005                                            NaN   
                    000006                                            NaN   
                    000008                                            NaN   
...                                                                   ...   
2021-05-31 09:30:00 688699                                            NaN   
                    688777                                       0.000267   
                    688788                                       0.000304   
                    688819                                       0.000078   
                    688981                                       0.000106   

                            COV_div_open_close_div_low_close_10_Shift_19  \
timestamp           ticker                                                 
2016-02-05 09:30:00 000001                                           NaN   
                    000004                                           NaN   
                    000005                                           NaN   
                    000006                                           NaN   
                    000008                                           NaN   
...                                                                  ...   
2021-05-31 09:30:00 688699                                           NaN   
                    688777                                      0.000093   
                    688788                                      0.000476   
                    688819                                      0.000120   
                    688981                                      0.000105   

                            COV_div_open_close_close_10_Shift_19  \
timestamp           ticker                                         
2016-02-05 09:30:00 000001                                   NaN   
                    000004                                   NaN   
                    000005                                   NaN   
                    000006                                   NaN   
                    000008                                   NaN   
...                                                          ...   
2021-05-31 09:30:00 688699                                   NaN   
                    688777                             -0.027520   
                    688788                             -0.026010   
                    688819                             -0.005039   
                    688981                             -0.006697   

                            COV_div_open_close_volume_sqrt_8_10_Shift_19  \
timestamp           ticker                                                 
2016-02-05 09:30:00 000001                                           NaN   
                    000004                                           NaN   
                    000005                                           NaN   
                    000006                                           NaN   
                    000008                                           NaN   
...                                                                  ...   
2021-05-31 09:30:00 688699                                           NaN   
                    688777                                      0.000337   
                    688788                                     -0.000738   
                    688819                                     -0.000232   
                    688981                                      0.000537   

                            COV_div_open_close_pct_chg_10_Shift_19  \
timestamp           ticker                                           
2016-02-05 09:30:00 000

In [6]:
final_df.dropna(axis=0,inplace=True)

In [7]:
final_df

COV_div_open_close_div_high_close_10_Shift_19  \
timestamp           ticker                                                  
2016-03-04 09:30:00 000001                                       0.000249   
                    000004                                       0.001573   
                    000005                                       0.001853   
                    000006                                       0.001297   
                    000008                                       0.001193   
...                                                                   ...   
2021-05-31 09:30:00 688689                                       0.000826   
                    688698                                       0.000441   
                    688777                                       0.000267   
                    688788                                       0.000304   
                    688981                                       0.000106   

                            COV_div_open_close_div_low_close_10_Shift_19  \
timestamp           ticker                                                 
2016-03-04 09:30:00 000001                                      0.000189   
                    000004                                      0.001082   
                    000005                                      0.002148   
                    000006                                      0.000805   
                    000008                                      0.001020   
...                                                                  ...   
2021-05-31 09:30:00 688689                                      0.000696   
                    688698                                      0.000623   
                    688777                                      0.000093   
                    688788                                      0.000476   
                    688981                                      0.000105   

                            COV_div_open_close_close_10_Shift_19  \
timestamp           ticker                                         
2016-03-04 09:30:00 000001                             -0.242335   
                    000004                             -0.156327   
                    000005                             -0.130646   
                    000006                             -0.397859   
                    000008                             -0.199992   
...                                                          ...   
2021-05-31 09:30:00 688689                             -0.026268   
                    688698                             -0.010271   
                    688777                             -0.027520   
                    688788                             -0.026010   
                    688981                             -0.006697   

                            COV_div_open_close_volume_sqrt_8_10_Shift_19  \
timestamp           ticker                                                 
2016-03-04 09:30:00 000001                                      0.000764   
                    000004                                      0.000258   
                    000005                                     -0.000738   
                    000006                                      0.001483   
                    000008                                     -0.001985   
...                                                                  ...   
2021-05-31 09:30:00 688689                                      0.000280   
                    688698                                     -0.002101   
                    688777                                      0.000337   
                    688788                                     -0.000738   
                    688981                                      0.000537   

                            COV_div_open_close_pct_chg_10_Shift_19  \
timestamp           ticker                                           
2016-03-04 09:30:00 000

In [8]:
AlphaNet.Data.convert_to_standard_daily_data_par(final_df,"AlphaNetV4_LSTM",data_path)

100%|██████████| 1276/1276 [57:36<00:00,  2.71s/it] 
